<img src="https://github.com/pmservice/ai-openscale-tutorials/raw/master/notebooks/images/banner.png" align="left" alt="banner">

# Working with Watson Machine Learning - Fairnes monitor and Explainibilty monitor

### Contents

- [1.0 Install Python Packages](#setup)
- [2.0 Configure Credentials](#credentials)
- [3.0 OpenScale configuration](#openscale)
- [4.0 Get Subscriptions](#subscription)
- [5.0 Fairness, drift monitoring and explanations](#fairness)

# 1.0 Install Python Packages

In [1]:
!rm -rf /home/spark/shared/user-libs/python3.6*

!pip install --upgrade ibm-ai-openscale==2.2.1 --no-cache | tail -n 1
!pip install --upgrade watson-machine-learning-client-V4==1.0.55 | tail -n 1
!pip install --upgrade pyspark==2.3 | tail -n 1

### Action: restart the kernel!

# 2.0 Configure credentials <a name="credentials"></a>



In [1]:
import warnings
warnings.filterwarnings('ignore')

The url for `WOS_CREDENTIALS` is the url of the CP4D cluster, i.e. `https://zen-cpd-zen.apps.com`. `username` and `password` must be ones that have cluster admin privileges.

In [2]:
WOS_CREDENTIALS = {
    "url": "https://zen-cpd-zen.omid-cp4d-v5-2bef1f4b4097001da9502000c44fc2b2-0001.us-south.containers.appdomain.cloud",
    "username": "<redacted>",
    "password": "<redacted>"
}

In [3]:
WML_CREDENTIALS = WOS_CREDENTIALS.copy()
WML_CREDENTIALS['instance_id']='openshift'
WML_CREDENTIALS['version']='2.5.0'

In [4]:
%store -r MODEL_NAME
%store -r DEPLOYMENT_NAME
%store -r DEFAULT_SPACE

# 3.0 Configure OpenScale <a name="openscale"></a>

The notebook will now import the necessary libraries and configure OpenScale

In [5]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient
import json

wml_client = WatsonMachineLearningAPIClient(WML_CREDENTIALS)

In [6]:
from ibm_ai_openscale import APIClient4ICP
from ibm_ai_openscale.engines import *
from ibm_ai_openscale.utils import *
from ibm_ai_openscale.supporting_classes import PayloadRecord, Feature
from ibm_ai_openscale.supporting_classes.enums import *

In [7]:
ai_client = APIClient4ICP(WOS_CREDENTIALS)
ai_client.version

'2.2.1'

# 4.0 Get Subscription <a name="subscription"></a>

In [8]:
subscription = None

if subscription is None:
    subscriptions_uids = ai_client.data_mart.subscriptions.get_uids()
    for sub in subscriptions_uids:
        if ai_client.data_mart.subscriptions.get_details(sub)['entity']['asset']['name'] == MODEL_NAME:
            print("Found existing subscription.")
            subscription = ai_client.data_mart.subscriptions.get(sub)
if subscription is None:
    print("No subscription found. Please run openscale-initial-setup.ipynb to configure.")

Found existing subscription.


### Set Deployment UID

In [9]:
wml_client.set.default_space(DEFAULT_SPACE)

'SUCCESS'

In [10]:
wml_deployments = wml_client.deployments.get_details()
deployment_uid = None
for deployment in wml_deployments['resources']:
    print(deployment['entity']['name'])
    if DEPLOYMENT_NAME == deployment['entity']['name']:
        deployment_uid = deployment['metadata']['guid']
        break
        
print(deployment_uid)

scottda Deployment 6-25
653f39ee-8e3b-4cde-8b38-2db5a1baf591


# 5.0 Fairness, drift monitoring and explanations 
 <a name="fairness"></a>

The code below configures fairness monitoring for our model. It turns on monitoring for two features, Sex and Age. In each case, we must specify:
  * Which model feature to monitor
  * One or more **majority** groups, which are values of that feature that we expect to receive a higher percentage of favorable outcomes
  * One or more **minority** groups, which are values of that feature that we expect to receive a higher percentage of unfavorable outcomes
  * The threshold at which we would like OpenScale to display an alert if the fairness measurement falls below (in this case, 95%)

Additionally, we must specify which outcomes from the model are favourable outcomes, and which are unfavourable. We must also provide the number of records OpenScale will use to calculate the fairness score. In this case, OpenScale's fairness monitor will run hourly, but will not calculate a new fairness rating until at least 200 records have been added. Finally, to calculate fairness, OpenScale must perform some calculations on the training data, so we provide the dataframe containing the data.

### Load the training data

In [11]:
!rm german_credit_data_biased_training.csv
!wget https://raw.githubusercontent.com/scottdangelo/credit-risk-workshop-cpd/addData/data/openscale/german_credit_data_biased_training.csv

    

rm: cannot remove 'german_credit_data_biased_training.csv': No such file or directory
--2020-06-25 17:10:44--  https://raw.githubusercontent.com/scottdangelo/credit-risk-workshop-cpd/addData/data/openscale/german_credit_data_biased_training.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 199.232.8.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|199.232.8.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 689622 (673K) [text/plain]
Saving to: 'german_credit_data_biased_training.csv'

100%[======================================>] 689,622     --.-K/s   in 0.02s   

2020-06-25 17:10:44 (30.8 MB/s) - 'german_credit_data_biased_training.csv' saved [689622/689622]



In [12]:
from pyspark.sql import SparkSession
import pandas as pd
import json

spark = SparkSession.builder.getOrCreate()
pd_data = pd.read_csv("german_credit_data_biased_training.csv", sep=",", header=0)
df_data = spark.read.csv(path="german_credit_data_biased_training.csv", sep=",", header=True, inferSchema=True)
df_data.head()

Row(CHECKINGSTATUS='0_to_200', LOANDURATION=31, CREDITHISTORY='credits_paid_to_date', LOANPURPOSE='other', LOANAMOUNT=1889, EXISTINGSAVINGS='100_to_500', EMPLOYMENTDURATION='less_1', INSTALLMENTPERCENT=3, SEX='female', OTHERSONLOAN='none', CURRENTRESIDENCEDURATION=3, OWNSPROPERTY='savings_insurance', AGE=32, INSTALLMENTPLANS='none', HOUSING='own', EXISTINGCREDITSCOUNT=1, JOB='skilled', DEPENDENTS=1, TELEPHONE='none', FOREIGNWORKER='yes', RISK='No Risk')

### 5.1 Enable fairness monitoring

In [13]:
subscription.fairness_monitoring.enable(
            features=[
                Feature("SEX", majority=['male'], minority=['female'], threshold=0.95),
                Feature("AGE", majority=[[26,75]], minority=[[18,25]], threshold=0.95)
            ],
            favourable_classes=['No Risk'],
            unfavourable_classes=['Risk'],
            min_records=200,
            training_data=pd_data
        )

## 5.2 Score the model again now that monitoring is configured

This next section randomly selects 200 records from the data feed and sends those records to the model for predictions. This is enough to exceed the minimum threshold for records set in the previous section, which allows OpenScale to begin calculating fairness.

In [14]:
!rm german_credit_feed.json
!wget https://raw.githubusercontent.com/scottdangelo/credit-risk-workshop-cpd/addData/data/openscale/german_credit_feed.json

rm: cannot remove 'german_credit_feed.json': No such file or directory
--2020-06-25 17:11:22--  https://raw.githubusercontent.com/scottdangelo/credit-risk-workshop-cpd/addData/data/openscale/german_credit_feed.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 199.232.8.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|199.232.8.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3076548 (2.9M) [text/plain]
Saving to: 'german_credit_feed.json'

100%[======================================>] 3,076,548   --.-K/s   in 0.06s   

2020-06-25 17:11:23 (48.9 MB/s) - 'german_credit_feed.json' saved [3076548/3076548]



Score 200 randomly chosen records

In [15]:
import random

with open('german_credit_feed.json', 'r') as scoring_file:
    scoring_data = json.load(scoring_file)

fields = scoring_data['fields']
values = []
for _ in range(200):
    values.append(random.choice(scoring_data['values']))
payload_scoring = {"fields": fields, "values": values}
payload = {
    wml_client.deployments.ScoringMetaNames.INPUT_DATA: [payload_scoring]
}
scoring_response = wml_client.deployments.score(deployment_uid, payload)

### 5.3 Run fairness monitor

Kick off a fairness monitor run on current data. The monitor runs hourly, but can be manually initiated using the Python client, the REST API, or the graphical user interface. We have a 180 second sleep so that the scoring of 200 payloads above can complete. NOTE: if the cell below finishes with errors, skip it and complete the notebook, then return and try again.

In [16]:
time.sleep(180)

run_details = subscription.fairness_monitoring.run(background_mode=False)




 Counting bias for deployment_uid=653f39ee-8e3b-4cde-8b38-2db5a1baf591 




FINISHED

---------------------------
 Successfully finished run 
---------------------------




In [17]:
time.sleep(5)

subscription.fairness_monitoring.show_table()

2020-06-25 17:14:31.013274+00:00,SEX,female,True,0.925,73.5,999,78338840-6e6c-4cf9-8293-a1893f607689,78338840-6e6c-4cf9-8293-a1893f607689,653f39ee-8e3b-4cde-8b38-2db5a1baf591,
2020-06-25 17:14:31.013274+00:00,AGE,"[18, 25]",False,1.07,84.0,999,78338840-6e6c-4cf9-8293-a1893f607689,78338840-6e6c-4cf9-8293-a1893f607689,653f39ee-8e3b-4cde-8b38-2db5a1baf591,
2020-06-25 17:12:13.795080+00:00,SEX,female,True,0.925,73.5,999,78338840-6e6c-4cf9-8293-a1893f607689,78338840-6e6c-4cf9-8293-a1893f607689,653f39ee-8e3b-4cde-8b38-2db5a1baf591,
2020-06-25 17:12:13.795080+00:00,AGE,"[18, 25]",False,1.07,84.0,999,78338840-6e6c-4cf9-8293-a1893f607689,78338840-6e6c-4cf9-8293-a1893f607689,653f39ee-8e3b-4cde-8b38-2db5a1baf591,


### 5.4 Configure Explainability

Finally, we provide OpenScale with the training data to enable and configure the explainability features.

In [18]:
from ibm_ai_openscale.supporting_classes import *
subscription.explainability.enable(training_data=pd_data)

In [19]:
explainability_details = subscription.explainability.get_details()

### 5.7 Run explanation for sample record

In [20]:
transaction_id = subscription.payload_logging.get_table_content(limit=1)['scoring_id'].values[0]

print(transaction_id)

78a67444-f896-463d-95ee-5b7efa451f38-1


In [21]:
explain_run = subscription.explainability.run(transaction_id=transaction_id, background_mode=False)




 Looking for explanation for 78a67444-f896-463d-95ee-5b7efa451f38-1 




in_progress...........................
finished

---------------------------
 Successfully finished run 
---------------------------




## Congratulations!

You have finished the hands-on lab for IBM Watson OpenScale Fairness and Explainability. You can now view the OpenScale dashboard by going to the CPD `Home` page, and clicking `Services`. Choose the `OpenScale` tile and click the menu to `Open`. Click on the tile for the model you've created to see fairness monitor. Click on the timeseries graph to get detailed information on transactions during a specific time window.

OpenScale shows model performance over time. You have two options to keep data flowing to your OpenScale graphs:
  * Download, configure and schedule the [model feed notebook](https://raw.githubusercontent.com/emartensibm/german-credit/master/german_credit_scoring_feed.ipynb). This notebook can be set up with your WML credentials, and scheduled to provide a consistent flow of scoring requests to your model, which will appear in your OpenScale monitors.
  * Re-run this notebook. Running this notebook from the beginning will delete and re-create the model and deployment, and re-create the historical data. Please note that the payload and measurement logs for the previous deployment will continue to be stored in your datamart, and can be deleted if necessary.

## Authors

Eric Martens, is a technical specialist having expertise in analysis and description of business processes, and their translation into functional and non-functional IT requirements. He acts as the interpreter between the worlds of IT and business.

Lukasz Cmielowski, PhD, is an Automation Architect and Data Scientist at IBM with a track record of developing enterprise-level applications that substantially increases clients' ability to turn data into actionable knowledge.

Zilu (Peter) Tang, is a cognitive developer with experties in deep learning and enterprise AI solutions from Watson Openscale to many other cutting-edge IBM research projects.